In [2]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import datetime as dt
from dateutil.relativedelta import relativedelta
import networkx as nx

pd.set_option('display.max_rows', 100)

In [5]:
#Return username from Tweet object
def userToUsername(blah,uname=''):
    try:
        len(blah)
        listOfUsernames = []
        for i in blah:
            listOfUsernames.append(i.username)
        return listOfUsernames        
    except:
        if blah == None or blah == '':
            return None
        else:
            if(blah.username == uname):
                return None
            else:
                return blah.username


#Function to retrieve tweet objects from a user, and return Pandas Dataframe of tweets and their corresponding features
def retrieveTweets(username,months=0,fromDate='',toDate='',maxTweets=20000):
    tweets_list1 = []

    if username == '':
        return print("You need to specify a username!")
    elif months != 0 and (fromDate != '' or toDate != ''):
        return print("You can only specify months, or from and to dates. Not both!")
    elif months != 0:
        mVar = months
        sinceDate = (dt.datetime.today() - relativedelta(months=mVar)).strftime("%Y-%m-%d") #"2015-01-01"
        toDate = dt.datetime.today().strftime("%Y-%m-%d")   #"2021-10-25"
    elif months == 0 and (fromDate =='' or toDate ==''):        
        return print("You need to specify months, or a fromDate and toDate!")
    else:
        sinceDate = fromDate
    unameinput = 'from:' + username
    print("Username: "+username)
    print("Months = " + str(months))
    print("From Date: "+sinceDate)
    print("To Date: " + toDate)
    print("Max Tweets: " + str(maxTweets))
    

    #Using TwitterSearchScraper to scrape data and append tweets to list
    for i,tweet in enumerate(sntwitter.TwitterSearchScraper(unameinput +" until:" + toDate +" since:"+sinceDate).get_items()):
        if i>maxTweets:
            break
        tweets_list1.append([tweet.date+ dt.timedelta(hours=2), tweet.id, tweet.content, tweet.user.username, userToUsername(tweet.inReplyToUser,username),userToUsername(tweet.mentionedUsers)])
    print("Qty of tweets = ",len(tweets_list1))

    #Creating a dataframe from the tweets list above
    tweets_df1 = pd.DataFrame(tweets_list1, columns=['Datetime', 'Tweet Id', 'Text', 'Username',"inReplyToUser","mentionedUsers"])
    return tweets_df1

In [ ]:
firstUser = ''

userHistory = []
tweetsToRetrieve = []
fDate = "2021-09-01"
tDate = "2021-10-11"


df = retrieveTweets(firstUser,fromDate=fDate,toDate=tDate,maxTweets=1000)
dfToSave = df.copy(deep = True)
topUsersRepliedTo = df['inReplyToUser'].value_counts()[0:13].index.tolist()
userHistory.append(firstUser)


for i in range(0,5):
    print("Iteration: " + str(i+1))
    print("<<<<<<<<<< >>>>>>>>>>")
    for username in topUsersRepliedTo:
        if username not in userHistory:
            UserDFTweetList = retrieveTweets(username,fromDate=fDate,toDate=tDate,maxTweets=2000)
            tweetsToRetrieve = UserDFTweetList['inReplyToUser'].value_counts()[0:13].index.tolist()
            userHistory.append(username)
            dfToSave = dfToSave.append(UserDFTweetList)
        else:
            print("User " + username + ' already scraped!')
        print('=======================')

    for username in tweetsToRetrieve:
        if username not in userHistory:
            UserDFTweetList = retrieveTweets(username,fromDate=fDate,toDate=tDate,maxTweets=2000)
            topUsersRepliedTo = UserDFTweetList['inReplyToUser'].value_counts()[0:13].index.tolist()
            userHistory.append(username)
            dfToSave = dfToSave.append(UserDFTweetList)
        else:
            print("User " + username + ' already scraped!')
        print('=======================')

In [12]:
fileNameSave = 'tweets.csv'
dfToSave.to_csv(fileNameSave)

In [ ]:
tweetDF = pd.read_csv(fileName)
tweetDF = tweetDF[['Username','inReplyToUser']]
listOfReplies = tweetDF[~tweetDF['inReplyToUser'].isnull()].values.tolist()

In [ ]:
G = nx.Graph()

for i in listOfReplies:
    if G.has_edge(i[0], i[1]):
        # we added this one before, just increase the weight by one
        #print("Edge Exists: "+str(i[0])+'<->'+str(i[1]))
        G[i[0]][i[1]]['weight'] += 1
    else:
        G.add_edge(i[0], i[1],weight=1)
        #print("Added edge: "+str(i[0])+'<->'+str(i[1]))

In [ ]:
nx.write_graphml(G,r'TwitterGraph.graphml')